# Train a Keras Sequential Model
## Using experiment
This notebook shows how to train a Keras Sequential model on SageMaker, and use SageMaker Experiment Management Python SDK to organize, track, compare, and evaluate your machine learning (ML) model training experiments.

You can track artifacts for experiments, including data sets, algorithms, hyper-parameters, and metrics. Experiments executed on SageMaker such as SageMaker Autopilot jobs and training jobs will be automatically tracked. You can also track artifacts for additional steps within an ML workflow that come before/after model training e.g. data pre-processing or post-training model evaluation.

The APIs also let you search and browse your current and past experiments, compare experiments, and identify best performing models.

The model used for this notebook is a simple deep CNN that was extracted from [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py). The experiment will be organized as follow:

1. Download and prepare the cifar10 dataset.
2. Train a Convolutional Neural Network (CNN) Model. Tune the hyper parameter that configures the optimization method in the model. Track the parameter configurations and resulting model accuracy using SageMaker Experiments Python SDK.
3. Finally use the search and analytics capabilities of Python SDK to search, compare and evaluate the performance of all model versions generated from model tuning in Step 2.
4. We will also see an example of tracing the complete linage of a model version i.e. the collection of all the data pre-processing and training configurations and inputs that went into creating that model version.

## The dataset
The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) is one of the most popular machine learning datasets. It consists of 60,000 32x32 images belonging to 10 different classes (6,000 images per class). Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

In this tutorial, we will train a deep CNN to recognize these images.


## Set up the environment

In [ ]:
import time
import os
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.analytics import ExperimentAnalytics

boto3_session = boto3.Session()
sm_client = boto3_session.client('sagemaker')
sm_session = sagemaker.Session(boto_session=boto3_session, sagemaker_client=sm_client)
role = get_execution_role()

In [ ]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [ ]:
bucket = '<BUCKET NAME>'
prefix = 'sagemaker/script-mode'

## Download the CIFAR-10 dataset
Downloading the test and training data takes around 5 minutes.

In [ ]:
#!pip install wget
# import wget # for TF2

#!python generate_cifar10_tfrecords_v1.x.py --data-dir data/
!mkdir data
!python generate_cifar10_tfrecords_v2.py --data-dir data/

## Run on SageMaker cloud

### Uploading the data to s3

In [ ]:
dataset_location = sm_session.upload_data(path='data', key_prefix='data/DEMO-cifar10-tf')
display(dataset_location)

Now lets track the parameters from the data pre-processing step.

In [ ]:
with Tracker.create(display_name="Preprocessing", sagemaker_boto_client=sm_client) as tracker:
    tracker.log_parameters({
        "datatype": 'tfrecords',
        "image_size": 32,
    })
    # we can log the s3 uri to the dataset we just uploaded
    tracker.log_input(name="cifar10-dataset", media_type="s3/uri", value=dataset_location)

SageMaker can get training metrics directly from the logs and send them to CloudWatch metrics.

In [ ]:
keras_metric_definition = [
    {'Name': 'train:loss', 'Regex': '.*loss: ([0-9\\.]+) - acc: [0-9\\.]+.*'},
    {'Name': 'train:accuracy', 'Regex': '.*loss: [0-9\\.]+ - acc: ([0-9\\.]+).*'},
    {'Name': 'validation:accuracy', 'Regex': '.*step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_acc: ([0-9\\.]+).*'},
    {'Name': 'validation:loss', 'Regex': '.*step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_acc: [0-9\\.]+.*'},
    {'Name': 'sec/steps', 'Regex': '.* - \d+s (\d+)[mu]s/step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_acc: [0-9\\.]+'}
]

### Step 1 - Set up the Experiment

Create an experiment to track all the model training iterations. Experiments are a great way to organize your data science work. You can create experiments to organize all your model development work for : [1] a business use case you are addressing, or [2] a data science team that owns the experiment, or [3] a specific data science and ML project. Think of it as a “folder” for organizing your “files”.

In [ ]:
cifar10_experiment = Experiment.create(
    experiment_name=f"cifar10-image-classification-{int(time.time())}", 
    description="Classification of images", 
    sagemaker_boto_client=sm_client)
print(cifar10_experiment)

### Step 2 - Track Experiment
### Now create a Trial for each training run to track the it's inputs, parameters, and metrics.
While training the CNN model on SageMaker, we will experiment with several values for optimization method in the model. We will create a Trial to track each training job run. We will also create a TrialComponent from the tracker we created before, and add to the Trial. This will enrich the Trial with the parameters we captured from the data pre-processing stage.

Note the execution of the following code takes a while. If you want to run the following training jobs asynchronously, you may need to increase your resource limit. Otherwise, you can run them sequentially.

In [ ]:
opt_method_trial_name_map = {}
for i, opt_method in enumerate(['adam','sgd','rmsprop']):
    # create trial
    trial_name = f"cifar10-training-job-with-{opt_method}-optimization-{int(time.time())}"
    cifar10_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=cifar10_experiment.experiment_name,
        sagemaker_boto_client=sm_client,
    )
    opt_method_trial_name_map[opt_method] = trial_name
    
    # associate the proprocessing trial component with the current trial
    cifar10_trial.add_trial_component(tracker.trial_component)


    estimator = TensorFlow(base_job_name='cifar10-tf',
                           entry_point='cifar10_keras_main.py',
                           source_dir=os.path.join(os.getcwd(), 'source_dir'),
                           role=role,
                           framework_version='1.12.0',
                           py_version='py3',
                           hyperparameters={'epochs': 1, 'batch-size' : 256, 'optimizer' : opt_method},
                           train_instance_count=1, train_instance_type='ml.p3.2xlarge',
                           metric_definitions=keras_metric_definition)
    
    cifar10_training_job_name = "cifar-training-job-{}".format(int(time.time()))
    remote_inputs = {'train' : dataset_location+'/train', 'validation' : dataset_location+'/validation', 'eval' : dataset_location+'/eval'}
    estimator.fit(remote_inputs, job_name=cifar10_training_job_name,
        experiment_config={
            "TrialName": cifar10_trial.trial_name,
            "TrialComponentDisplayName": "Training",
        },
        wait=False,)
    # give it a while before dispatching the next training job
    time.sleep(2)

### Compare the model training runs for an experiment

Now we will use the analytics capabilities of Python SDK to query and compare the training runs for identifying the best model produced by our experiment. You can retrieve trial components by using a search expression.

### Some Simple Analyses

In [ ]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

In [ ]:
trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sm_session, 
    experiment_name=cifar10_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.validation:accuracy.max",
    sort_order="Descending",
    metric_names=['train:accuracy', 'validation:accuracy'],
    parameter_names=['optimizer']
)

In [ ]:
trial_component_analytics.dataframe()

Next let's look at an example of tracing the lineage of a model by accessing the data tracked by SageMaker Experiments for a `cifar-training-job` trial

In [ ]:
lineage_table = ExperimentAnalytics(
    sagemaker_session=sm_session, 
    search_expression={
        "Filters":[{
            "Name": "Parents.TrialName",
            "Operator": "Equals",
            "Value": opt_method_trial_name_map['adam']
        }]
    },
    sort_by="CreationTime",
    sort_order="Ascending",
)
lineage_table.dataframe()